Model Training start

In [20]:
# Import library
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Modelling
from sklearn.metrics import mean_squared_error,r2_score
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor,AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression,Ridge,Lasso
from sklearn.metrics import r2_score,mean_absolute_error,mean_absolute_error
from sklearn.model_selection import RandomizedSearchCV
from xgboost import XGBRegressor
# Import warnings 
import warnings
warnings.filterwarnings('ignore')

In [21]:
# Import the dataset
df=pd.read_csv('data/stud.csv')
df.head()

,gender,race_ethnicity,parental_level_of_education,lunch,test_preparation_course,math_score,reading_score,writing_score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44
4,male,group C,some college,standard,none,76,78,75


In [22]:
X = df.drop('math_score', axis=1)
y = df['math_score']


In [23]:
X.head()

,gender,race_ethnicity,parental_level_of_education,lunch,test_preparation_course,reading_score,writing_score
0,female,group B,bachelor's degree,standard,none,72,74
1,female,group C,some college,standard,completed,90,88
2,female,group B,master's degree,standard,none,95,93
3,male,group A,associate's degree,free/reduced,none,57,44
4,male,group C,some college,standard,none,78,75


In [24]:
y.head()

0    72
1    69
2    90
3    47
4    76
Name: math_score, dtype: int64

In [25]:
# categorical and numerical feature
categorical_feature=[feature for feature in df.columns if df[feature].dtype =='object']
numerical_feature=[feature for feature in df.columns if df[feature].dtype !='O']
print('categorical_feature :-->',categorical_feature)
print('numerical_feature :-->',numerical_feature)

categorical_feature :--> ['gender', 'race_ethnicity', 'parental_level_of_education', 'lunch', 'test_preparation_course']
numerical_feature :--> ['math_score', 'reading_score', 'writing_score']


In [26]:
for cat in categorical_feature:
    print(f"categorical in {cat}\n{df[cat].unique()}\n{'='*100}")


categorical in gender
['female' 'male']
categorical in race_ethnicity
['group B' 'group C' 'group A' 'group D' 'group E']
categorical in parental_level_of_education
["bachelor's degree" 'some college' "master's degree" "associate's degree"
 'high school' 'some high school']
categorical in lunch
['standard' 'free/reduced']
categorical in test_preparation_course
['none' 'completed']


In [27]:
num_feature = X.select_dtypes(exclude='object').columns
cat_feature = X.select_dtypes(include='object').columns
from sklearn.preprocessing import OneHotEncoder,StandardScaler
from sklearn.compose import ColumnTransformer
numerical_scaler = StandardScaler()
categorical_encoder = OneHotEncoder()

preprocessor = ColumnTransformer(
    [
        ("Onehotencoding", categorical_encoder, cat_feature),
        ("Standardscaler1", numerical_scaler, num_feature)
    ]
)



In [28]:
X_transformed = preprocessor.fit_transform(X)

In [30]:
X_transformed.shape


(1000, 19)

In [31]:
# Train to split
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X_transformed,y,test_size=0.2,random_state=42)
X_train.shape,X_test.shape,y_train.shape,y_test.shape

((800, 19), (200, 19), (800,), (200,))

In [32]:
# Create the function that hep to evaluated to give metric for all the models
def evaluate_models(true,predicted):
    mae=mean_absolute_error(true,predicted)
    mse=mean_squared_error(true,predicted)
    rmse=np.sqrt(mean_absolute_error(true,predicted))
    r2_square=r2_score(true,predicted)
    return mae,rmse,r2_square

In [33]:
models={
    "LinearRegression":LinearRegression(),
    "Lasso":Lasso(),
    "Ridge":Ridge(),
    "KNeighborsRegressor":KNeighborsRegressor(),
    "DecisionTreeRegressor":DecisionTreeRegressor(),
    "XGBRegressor":XGBRegressor(),
    "AdaBoostRegressor":AdaBoostRegressor()
}

In [34]:
model_list = []
r2_list = []

for model_name, model in models.items():
    model.fit(X_train, y_train)

    # Make predictions
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)

    # Evaluation on train and test data
    model_train_mae, model_train_rmse, model_train_r2 = evaluate_models(
        y_train, y_train_pred
    )
    model_test_mae, model_test_rmse, model_test_r2 = evaluate_models(
        y_test, y_test_pred
    )

    print(f"Model name is : {model_name}")
    print(f"Train MAE  : {model_train_mae}")
    print(f"Train RMSE : {model_train_rmse}")
    print(f"Train R2   : {model_train_r2}")
    print(f"Test MAE   : {model_test_mae}")
    print(f"Test RMSE  : {model_test_rmse}")
    print(f"Test R2    : {model_test_r2}")
    print("=" * 50)

    model_list.append(model_name)
    r2_list.append(model_test_r2)


Model name is : LinearRegression
Train MAE  : 4.266711846071957
Train RMSE : 2.065602054141106
Train R2   : 0.8743172040139593
Test MAE   : 4.21476314247485
Test RMSE  : 2.0529888315514166
Test R2    : 0.8804332983749565
Model name is : Lasso
Train MAE  : 5.206302661246526
Train RMSE : 2.2817323816009902
Train R2   : 0.8071462015863456
Test MAE   : 5.157881810347763
Test RMSE  : 2.2710970499623664
Test R2    : 0.8253197323627853
Model name is : Ridge
Train MAE  : 4.264987823725981
Train RMSE : 2.0651846948217445
Train R2   : 0.8743042615212909
Test MAE   : 4.211100688014261
Test RMSE  : 2.0520966565964334
Test R2    : 0.8805931485028737
Model name is : KNeighborsRegressor
Train MAE  : 4.516749999999999
Train RMSE : 2.1252646893975347
Train R2   : 0.8554978341651085
Test MAE   : 5.621
Test RMSE  : 2.3708648211148606
Test R2    : 0.7838129945787431
Model name is : DecisionTreeRegressor
Train MAE  : 0.01875
Train RMSE : 0.13693063937629152
Train R2   : 0.9996534669718089
Test MAE   : 6.44

In [36]:
pd.DataFrame(list(zip(model_list,r2_list)),columns=['model Name','R2_score']).sort_values(by=['R2_score'],ascending=False)

,model Name,R2_score
2,Ridge,0.880593
0,LinearRegression,0.880433
6,AdaBoostRegressor,0.847430
5,XGBRegressor,0.827797
1,Lasso,0.825320
3,KNeighborsRegressor,0.783813
4,DecisionTreeRegressor,0.728937
